In [1]:
#Import the required libraries and modules that you would need.
import pandas as pd
## plotting libraries
from matplotlib import pyplot as plt
## Sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,  ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
%matplotlib inline 

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read that data into Python and call the dataframe churnData.
customer_churn = pd.read_csv("file:///Users/genevauy/Desktop/DATA_Customer-Churn.csv")
customer_churn

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [3]:
customer_churn['TotalCharges'] = pd.to_numeric(customer_churn['TotalCharges'], downcast = 'float', errors = 'coerce')
customer_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7032 non-null   float32
 15  Churn             7043 non-null   object 
dtypes: float32(1), float64(1), int64(2), objec

In [4]:
mean_totalcharges = customer_churn['TotalCharges'].mean()
customer_churn['TotalCharges'] = customer_churn['TotalCharges'].fillna(mean_totalcharges)

In [5]:
selected_columns = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges','Churn']  # Replace with the actual column names you want to select
df = customer_churn[selected_columns]

In [6]:
df = df.rename(columns=lambda x: x.lower())

In [7]:
df["churn"] = df["churn"].apply(lambda x: x.replace('No', '0') if type(x) == str else x)
df["churn"] = df["churn"].apply(lambda x: x.replace('Yes', '1') if type(x) == str else x)

In [8]:
df['churn']= df['churn'].astype(int)

In [9]:
df

,tenure,seniorcitizen,monthlycharges,totalcharges,churn
0,1,0,29.85,29.850000,0
1,34,0,56.95,1889.500000,0
2,2,0,53.85,108.150002,1
3,45,0,42.30,1840.750000,0
4,2,0,70.70,151.649994,1
...,...,...,...,...,...
7038,24,0,84.80,1990.500000,0
7039,72,0,103.20,7362.899902,0
7040,11,0,29.60,346.450012,0
7041,4,1,74.40,306.600006,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tenure          7043 non-null   int64  
 1   seniorcitizen   7043 non-null   int64  
 2   monthlycharges  7043 non-null   float64
 3   totalcharges    7043 non-null   float32
 4   churn           7043 non-null   int64  
dtypes: float32(1), float64(1), int64(3)
memory usage: 247.7 KB


#### splitting

In [11]:
X = df[["tenure","seniorcitizen","monthlycharges","totalcharges"]]
y = df[['churn']]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

#### balancing

In [13]:
y.value_counts()

churn
0        5174
1        1869
dtype: int64

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [15]:
conda install -c conda-forge imbalanced-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.1



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [29]:
X, y = smote.fit_resample(X_train, y_train)
y.value_counts()

churn
0        4143
1        4143
dtype: int64

#### scaling

In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [42]:
X_scaler_train= scaler.transform(X_train)

X_scaler_test= scaler.transform(X_test)

### Decision Tree Classification

In [43]:
TT_SPLIT = 0.2     # ratio train/test size
RAND_STATE = 42

In [44]:
model = DecisionTreeClassifier(max_depth =5)
model.fit(X_train, y_train)
print("The (mean) accuracy on the test set is %.2f" %(model.score(X_test, y_test)))
print("The (mean) accuracy on the train data is %.2f" %(model.score(X_train, y_train)))
print("X has %d rows and %d columns"  %(X.shape[0],X.shape[1]))
print("y has %d rows"  %(y.shape[0]))

The (mean) accuracy on the test set is 0.79
The (mean) accuracy on the train data is 0.80
X has 8286 rows and 4 columns
y has 8286 rows


In [45]:
scores=cross_val_score(model, X_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.76929902 0.78881988 0.78704525 0.76929902 0.78952043]
Score stats: 0.78 accuracy with a standard deviation of 0.01


In [35]:
print("Five-fold cv results: \n %0.2f mean accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Five-fold cv results: 
 0.78 mean accuracy with a standard deviation of 0.01


In [36]:
y_test

,churn
6625,1
5975,1
6891,0
1681,0
4780,0
...,...
6525,0
710,1
6639,0
6844,0


In [37]:
y_pred = cross_val_predict(model, X_test, y_test, cv=5)
y_pred

array([1, 1, 0, ..., 0, 0, 0])

In [46]:
model1 = DecisionTreeClassifier(max_depth=5)
model2 = LogisticRegression()
model3 = KNeighborsClassifier()

model_pipeline = [model1, model2, model3]
model_names = ['Classification Tree', 'Logistic Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Classification Tree': 0.7807967205725445, 'Logistic Regression': 0.7905580921070258, 'KNN': 0.766949933885053}


In [47]:
print("Comparing the 3 regression scores we find \n")

pd.DataFrame([scores], index=["score"])

Comparing the 3 regression scores we find 



,Classification Tree,Logistic Regression,KNN
score,0.780797,0.790558,0.76695


In [48]:
y_train.value_counts()

churn
0        4143
1        1491
dtype: int64